In [1]:
# find ACMI director creators and attempt matching titles with Wikipedia full-text.

import json
import numpy
import pandas
import pathlib
import pydash
import requests
import tqdm

acmi_path = pathlib.Path.cwd().parents[0] / 'acmi-api' / 'app' / 'json' / 'works'
acmi_files = [filename for filename in acmi_path.iterdir() if filename.suffix == '.json' and 'index' not in filename.name]

directors = list()

for acmi_file in tqdm.tqdm(acmi_files):

    with open(acmi_file, encoding='utf-8') as acmi_data:
        acmi_data = json.load(acmi_data)

    if 'creators_primary' in acmi_data:
        for y in acmi_data['creators_primary']:
            if 'role' in y:
                if y['role'] == 'director':
                    directors.append(y['creator_id'])

print(len(pydash.uniq(directors)))

acmi_creators = pandas.read_csv(pathlib.Path.cwd().parents[0] / 'acmi-api' / 'app' / 'tsv' / 'creators.tsv', delimiter='\t', low_memory=False)
acmi_creators = acmi_creators.loc[acmi_creators.id.isin(directors)]

print(len(acmi_creators))
acmi_creators.sample(10)

100%|██████████| 42921/42921 [00:42<00:00, 1002.80it/s]


7548
3497


,id,name,also_known_as,date_of_birth,date_of_death,places_of_operation,biography,biography_author,date_of_biography,external_links,uuid,source,source_identifier,external_references,date_modified
16891,24056,Graeme MacLeod,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-09T19:29:59.498435+10:00,NaN,123a6d96-66be-4c29-a5ae-3487283c5697,Vernon,19977.0,NaN,2023-06-14T16:33:30.576250+10:00
16937,77235,Alan Johnson,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-09T17:42:35.018405+10:00,NaN,b19dd6e2-72b7-4a2f-a586-075363bd2046,Vernon,14770.0,NaN,2023-06-14T16:33:30.576250+10:00
16014,10915,Roberto Salvatore,NaN,NaN,NaN,NaN,NaN,NaN,2023-07-05T03:55:46.063535+10:00,NaN,0d1f12e6-0f2c-43bd-9beb-3622264d8d26,Vernon,2666.0,NaN,2023-07-05T03:55:46.063866+10:00
14498,79292,J. Elder Wills,NaN,NaN,NaN,United Kingdom,NaN,NaN,2023-05-09T16:21:25.463742+10:00,https://en.wikipedia.org/wiki/J._Elder_Wills,1d751566-26ed-4f83-bc35-867247e11a4b,Vernon,20963.0,"(Wikidata,Q3161060),(VIAF,92820683)",2023-06-14T16:33:30.576250+10:00
6515,6777,Roman Stolz,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-09T20:00:21.826414+10:00,NaN,eebb0941-7692-42ee-894a-7f1d2323b3db,Vernon,110666.0,NaN,2023-06-14T16:33:30.576250+10:00
14371,12617,Paul Burnford,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-09T15:14:03.284882+10:00,NaN,7e70e223-c885-423a-a76b-eb6679220f88,Vernon,1159.0,NaN,2023-06-14T16:33:30.576250+10:00
8444,75005,Chips Rafferty,NaN,26 MAR 1909,27 MAY 1971,NaN,NaN,NaN,2023-05-09T18:22:37.257056+10:00,NaN,6b7c48df-d5d6-48f5-ad09-fde35a5daf23,Vernon,16791.0,NaN,2023-06-14T16:33:30.576250+10:00
7172,83306,Jean-Pierre Decourt,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-09T12:35:13.277377+10:00,NaN,28246632-c4fc-4fdb-9405-73fbf1c492f0,Vernon,31321.0,NaN,2023-06-14T16:33:30.576250+10:00
7407,21792,Catherine South,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-09T15:53:10.140904+10:00,NaN,d4196b71-e888-4d11-84d9-ddae47b18226,Vernon,16604.0,NaN,2023-06-14T16:33:30.576250+10:00
3936,81731,Anita Sterner,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-09T18:37:29.121252+10:00,NaN,8e550bc0-4265-4932-aa83-68b7ca114f36,Vernon,15282.0,NaN,2023-06-14T16:33:30.576250+10:00
